# Bag Of Words Topic Modeling and Recommender for Ted Talks - JBCS
 - Summer K. Rankin 

once the text has been cleaned, we move onto the next steps
        - 1. Vectorize 
        - 2. Topic modeling
        - 3. Visualization
        - 4. Recommender (optional)

# Load the packages

In [17]:
import nltk, re, pickle, os
import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation,  TruncatedSVD, NMF
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing  import  StandardScaler

import seaborn as sns
%matplotlib inline
import matplotlib.pyplot as plt

import pyLDAvis, pyLDAvis.sklearn
from IPython.display import display

# Import the cleaned text

In [3]:
food = pd.read_csv('./data/Food_Inspections.csv', encoding = "UTF-8")  

In [4]:
corpus = food.Violations.dropna().reset_index(drop=True)


# 1 Vectorization
Vectorization is the important step of turning our words into numbers. There are 2 common methods: count vectorizer, tf-idf. This function takes each word in each document and counts the number of times the word appears. You end up with each word (and n-gram) as your columns and each row is a document, so the data is the frequency of each word in each document. As you can imagine, there will be a large number of zeros in this matrix; we call this a sparse matrix. https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html

tf-IDF

https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html#sklearn.feature_extraction.text.TfidfVectorizer

----

For this tutorial the tokenization and vectorization gets bundled together because we are using skleanrn's feature extraction functions. This means we will set the parameters of these functions to tokenize the way we want, include n-grams, and set thresholds for max or min document frequency of a term. https://towardsdatascience.com/hacking-scikit-learns-vectorizers-9ef26a7170af

- takes us from words to numbers 
- create the document-term matrix which is the basis for all modeling
    - row = document, column = word or n-gram, data = word's weight for that document
- we will vectorize in 2 ways 
    1. counting the frequency of each term in each document (**CountVectorizer**)
    2. counting the frequency of each term in each document and weighting by the number of times the term appears in the corpus. Term Frequency * Inverse Document Frequency (**TfidfVectorizer**)


# 1.1 Count Vectorize 
+ Using Sklearn algorithms with text data
+ CountVectorizer: Convert a collection of text documents to a matrix of token counts 
+ This implementation produces a sparse representation
+  **CountVectorizer** is a class; so **vectorizer** below represents an instance of that object
+ note that we can also **lowercase** , remove **stopwords** and search for a certain pattern **token_pattern** (i.e. letters only) by using the parameters of this function. 
+ a specific vocabulary can also be passed to this function. 
+ for more info about the many parameters, see the sklearn docs. 

https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html

CountVectorizer is a class; so `vectorizer` below represents an instance of that object.




In [5]:
c_vectorizer = CountVectorizer(max_df = 0.6, 
                             max_features=2000)

# call `fit` to build the vocabulary and calculate the weights
c_vectorizer.fit(corpus)

# finally, call `transform` to apply the weights and convert text to a bag of words
count_vect_data = c_vectorizer.transform(corpus)

# to view the document-term matrix, we can transpose back to a dense array
pd.DataFrame(data = count_vect_data.toarray(), columns=sorted(c_vectorizer.get_feature_names()))

,00,005,005a,01,010,010a,010b,012,012a,02,...,worned,worth,wrap,wrapped,written,yard,yellow,yogurt,you,your
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,4,5,0,0,1,0,0,0,0,0,...,0,0,0,0,3,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
146959,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
146960,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
146961,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
146962,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


### Pre-processing with sklearn
Parameters in the sklearn vectorizer class that can be used to do the cleaning and normalization of text (can be used as an alternative to what we did in the first notebook if the processing is light:<br>
* `analyzer='word'`: Tokenize by word
* `ngram_range=(1,3)`: Keep all 1 and 2, and 3-word grams
* `stop_words='english'`: Remove all English stop words
* `token_pattern='\\b[a-z][a-z]+\\b'`: Match all tokens with 2 or more (strictly) alphabet characters

TODO: import the original ted_talks and pre-process them using skleanrn methods instead of writing your own function. 


In [8]:
c_vectorizer_skl = CountVectorizer(analyzer='word', 
                                   ngram_range=(1,3),
                                   stop_words='english',
                                   token_pattern='\\b[a-z][a-z]+\\b',
                                    max_df = 0.6, 
                                    max_features=2000)

# call `fit` to build the vocabulary and calculate the weights
c_vectorizer_skl.fit(corpus)

# finally, call `transform` to apply the weights and convert text to a bag of words
count_vect_data_skl = c_vectorizer_skl.transform(corpus)

# to view the document-term matrix, we can transpose back to a dense array
pd.DataFrame(data = count_vect_data_skl.toarray(), columns=sorted(c_vectorizer_skl.get_feature_names()))

,abrasive,abrasive detergents,abrasive detergents comments,absorbent,accessible,accessible comments,accessible comments observed,accessible designed,accessible designed maintained,accessible food,...,wiping cloths,women,wood,work,working,worn,worn comments,written,written log,written log shall
0,1,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,3,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
146959,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
146960,1,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
146961,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
146962,1,1,1,0,2,0,0,1,1,0,...,0,0,0,0,1,0,0,0,0,0


# 1.2 Tf-idf (term frequency inverse document frequency)
+ gives more weight to less frequent terms 

by calculating a weight for each term $j$ in each document $i$ 

$$
w_{i,j}= tf_{i,j} x \log\frac{N}{df_j}
$$

$N$ = total documents
$j$ = term
$i$ = document
$df$ = document frequency
$tf$ = term frequency

frequency of the $j$th term in the $i$th document multiplied by the log of the total documents divided by the number of documents in the corpus containing the $j$th term. 


https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html#sklearn.feature_extraction.text.TfidfVectorizer

In [10]:
t_vectorizer = TfidfVectorizer(analyzer='word', 
                                   ngram_range=(1,3),
                                   stop_words='english',
                                   token_pattern='\\b[a-z][a-z]+\\b',
                                    max_df = 0.6, 
                                    max_features=2000)


# call `fit` to build the vocabulary and calculate weights
t_vectorizer.fit(corpus)

# finally, call `transform` to convert text to a bag of words
tfidf_data = t_vectorizer.transform(corpus)

In [11]:
# view a dense representation of the document-term matrix
pd.DataFrame(tfidf_data.toarray(), columns=t_vectorizer.get_feature_names())

,abrasive,abrasive detergents,abrasive detergents comments,absorbent,accessible,accessible comments,accessible comments observed,accessible designed,accessible designed maintained,accessible food,...,wiping cloths,women,wood,work,working,worn,worn comments,written,written log,written log shall
0,0.033922,0.033922,0.033940,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0000,0.0,0.0
1,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0000,0.0,0.0
2,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0616,0.0,0.0
3,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0000,0.0,0.0
4,0.000000,0.000000,0.000000,0.0,0.138501,0.179583,0.0,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
146959,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0000,0.0,0.0
146960,0.082970,0.082970,0.083015,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0000,0.0,0.0
146961,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0000,0.0,0.0
146962,0.020996,0.020996,0.021008,0.0,0.076204,0.000000,0.0,0.053091,0.053091,0.0,...,0.0,0.0,0.0,0.0,0.043593,0.0,0.0,0.0000,0.0,0.0


# 2 Topic modeling 
Use the document term matrix created with vectorization, to create a latent space and find the words that tend to ocurr together

We will use LDA Latent Dirichlet Allocation here (there are several methods, NMF, SVD)

This will reduce the data from thousands of terms (dimensions) to 20 topics. (Dimensionality Reduction)

Creates a latent space that is X dimensions.

https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.LatentDirichletAllocation.html

# LDA Latent Dirichlet Allocation

https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.LatentDirichletAllocation.html

In [12]:
def topic_mod(vectorizer, vect_data, topics=20, iters=5, no_top_words=50):
    
    """ use Latent Dirichlet Allocation to get topics"""

    mod = LatentDirichletAllocation(n_components=topics,
                                    max_iter=iters,
                                    random_state=42,
                                    learning_method='online',
                                    n_jobs=-1)
    
    mod_dat = mod.fit_transform(vect_data)
    
    
    # to display a list of topic words and their scores 
    
    def display_topics(model, feature_names, no_top_words):
        for ix, topic in enumerate(model.components_):
            print("Topic ", ix)
            print(" ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]) + '\n')
    
    display_topics(mod, vectorizer.get_feature_names() , no_top_words)

    
    return mod, mod_dat

#### TODO: Calculate the topic modeling using both of the count vectorized methods and tfidf to compare differences

In [20]:
# Count Vectorizer 
c_lda_model, c_lda_data = topic_mod(c_vectorizer, 
                            count_vect_data, 
                            topics=40, 
                            iters=10, 
                            no_top_words=15)  

Topic  0
floors free detergents abrasive utensils 33 cleaned 34 prep coving maintain all detail interior area

Topic  1
prep on area from replace or door floor cooler wall units shelving walk rear damaged

Topic  2
violation corrected previous 42 report 090 minor not from inspection violations on summary all dated

Topic  3
inspector meat walk cooler deli display produce bakery codes sales cutting case slicer coolers butcher

Topic  4
sink compartment water three area designed exposed hand drain serious under dish provide at prep

Topic  5
thermometers metal stem provided refrigeration conspicuous 40 provide thermometer for coolers cooler units inside all

Topic  6
be shall all cleaned easily utensils smooth kept floors with sanitized thoroughly or storage establishment

Topic  7
city chicago certificate manager instructed site sanitation all on at service inspection that allergen person

Topic  8
stored articles unnecessary premises litter 41 storage remove all floor clutter off rear 

In [22]:
# TF-IDF Vectorizer 
t_lda_model, t_lda_data = topic_mod(t_vectorizer, 
                            tfidf_data, 
                            topics=40, 
                            iters=10, 
                            no_top_words=15)  


Topic  0
certified manager served certified food hazardous potentially potentially hazardous prepared served prepared food manager hazardous foods certified food manager potentially hazardous foods manager site foods

Topic  1
shall vented plumbing easily comments food parts utensils shall shall good shall good repair shall smooth shall smooth easily cleanable durable easily cleanable durable durable shall durable shall good

Topic  2
written log shall exposed events events comments events comments observed events instructed evidence evidence rodent evidence rodent insect excess excessive excessive grease exhaust exit exit door

Topic  3
comments instructed comments instructed clean instructed summary instructed clean instructed clean maintain maintained comments instructed used comments instructed detergents comments instructed visible report summary inspection report summary clean maintain stored comments instructed inspection report

Topic  4
backflow prevention devices backflow pre

In [ ]:
# TF-IDF Vectorizer on raw talks 


# 3 Visualization  

# 3.1 View distribution of topics with pyLDAvis 
+ plot the first 2 components from the topic modeling (LDA). 
+ Not really the best way to look at clusters, but a good place to start and a very nice way to present data to clients

TODO: also plot the visualization for the raw vs. cleaned (by you) 
and count vectorizled vs. tf-idf vectorized

In [24]:

# Setup to run in Jupyter notebook
pyLDAvis.enable_notebook()

 # Create the visualization
vis = pyLDAvis.sklearn.prepare(t_lda_model, tfidf_data, t_vectorizer, sort_topics=False, mds='mmds')

 # can export as a standalone HTML web page
pyLDAvis.save_html(vis, 'lda_visual.html')

# # Let's view it!
display(vis)

/anaconda3/envs/jbcs2020/lib/python3.6/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.367008  0.106967       1        1   3.455002
1      0.189229  0.080287       2        1   6.917109
2      0.068217 -0.017623       3        1   0.222357
3     -0.347418 -0.349948       4        1   0.860189
4      0.133280 -0.496616       5        1   0.641160
5      0.096289 -0.224035       6        1   3.999966
6      0.068218 -0.017623       7        1   0.222357
7      0.564915 -0.102782       8        1   0.942733
8     -0.149733 -0.013909       9        1   1.176431
9      0.195567 -0.268770      10        1   9.403779
10    -0.436532  0.083258      11        1   2.559327
11    -0.016545 -0.444060      12        1   3.236411
12     0.250234 -0.173251      13        1   1.343382
13     0.232009  0.471526      14        1   0.294812
14     0.015750  0.145571      15        1   4.437465
15    -0.153093 -0.255889      16        1   1.043507
16    -0.191849  0.466993      17        1   1.084013
17     0.397527  0.189021      18        1   2.472245
18     0.068218 -0.017623      19        1   0.222357
19    -0.494569 -0.074231      20        1   3.550929
20    -0.039928  0.481138      21        1   0.588302
21    -0.032896 -0.342615      22        1   1.054373
22    -0.236570 -0.197952      23        1   1.522931
23    -0.277664  0.010550      24        1   1.581637
24     0.078352  0.492098      25        1   0.453107
25     0.386059  0.013968      26        1   0.749437
26     0.411726 -0.223778      27        1   0.541248
27    -0.426431  0.203182      28        1   0.667477
28    -0.355984 -0.052675      29        1   0.828065
29     0.250080  0.258542      30        1   4.625175
30     0.376059  0.374829      31        1   0.392386
31    -0.311559  0.389953      32        1   1.064605
32     0.023542  0.284214      33        1   3.245812
33     0.337708 -0.394192      34        1   0.583299
34     0.068217 -0.017623      35        1   0.222357
35    -0.200341 -0.507241      36        1   1.749109
36    -0.074197 -0.137589      37        1  24.788741
37    -0.392213 -0.218442      38        1   4.988313
38    -0.312489  0.253018      39        1   0.546978
39    -0.128191  0.243353      40        1   1.721116, topic_info=     Category         Freq        Term        Total  loglift  logprob
965   Default  7985.000000  instructed  7985.000000  30.0000  30.0000
599   Default  2491.000000      easily  2491.000000  29.0000  29.0000
1221  Default  5919.000000    observed  5919.000000  28.0000  28.0000
1710  Default  2233.000000      smooth  2233.000000  27.0000  27.0000
1083  Default  5920.000000    maintain  5920.000000  26.0000  26.0000
...       ...          ...         ...          ...      ...      ...
1693  Topic40   427.709622       sinks  1421.476821   2.8612  -4.0937
20    Topic40   371.393044    adequate  1257.931523   2.8422  -4.2349
1971  Topic40   406.362701     washing  1602.908775   2.6899  -4.1449
904   Topic40   313.416684   hot water   939.722819   2.9641  -4.4046
1682  Topic40   380.951852        sink  4801.112546   1.5283  -4.2095

[1796 rows x 6 columns], token_table=      Topic      Freq               Term
term                                    
0         1  0.010809           abrasive
0         2  0.036998           abrasive
0         3  0.000831           abrasive
0         6  0.016628           abrasive
0         7  0.000831           abrasive
...     ...       ...                ...
1999      3  0.002496  written log shall
1999      7  0.002496  written log shall
1999     10  0.990927  written log shall
1999     19  0.002496  written log shall
1999     35  0.002496  written log shall

[12807 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40])

In [21]:
# TODO plot the tf-idf vectorization
vis = pyLDAvis.sklearn.prepare(c_lda_model, count_vect_data, c_vectorizer, sort_topics=False, mds='mmds')

 # can export as a standalone HTML web page
pyLDAvis.save_html(vis, 'lda_visual.html')

# # Let's view it!
display(vis)

/anaconda3/envs/jbcs2020/lib/python3.6/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
0      0.010304  0.353742       1        1  9.070803
1     -0.081059  0.220119       2        1  3.137381
2     -0.437167 -0.192895       3        1  1.481532
3      0.101811  0.550716       4        1  0.647748
4     -0.095218 -0.013188       5        1  3.325272
5      0.389814 -0.136117       6        1  1.712221
6      0.214341  0.375463       7        1  6.359909
7      0.105153 -0.353818       8        1  1.057726
8     -0.345227  0.235202       9        1  3.814747
9      0.296787 -0.037902      10        1  1.115970
10     0.113221  0.176846      11        1  3.326210
11     0.367119  0.190021      12        1  0.674352
12    -0.261333  0.391418      13        1  1.472457
13    -0.233699  0.246411      14        1  0.708628
14     0.149215 -0.324886      15        1  2.565076
15    -0.156644  0.441287      16        1  3.503174
16    -0.482800 -0.046789      17        1  3.971656
17    -0.179142  0.065977      18        1  5.768250
18     0.094265  0.300409      19        1  1.582766
19     0.248255  0.101784      20        1  1.520994
20    -0.301272 -0.262790      21        1  0.954407
21    -0.025377 -0.497869      22        1  1.179130
22     0.360780  0.276768      23        1  2.184853
23    -0.050169 -0.245397      24        1  1.596442
24    -0.348416  0.060612      25        1  3.897737
25    -0.097614 -0.141213      26        1  2.656878
26     0.564310 -0.056548      27        1  0.328012
27    -0.424524  0.182311      28        1  1.320808
28     0.202469 -0.412126      29        1  1.269922
29    -0.178593 -0.436995      30        1  1.554229
30    -0.304434 -0.069363      31        1  1.928575
31     0.079919 -0.090232      32        1  5.265404
32    -0.135390 -0.309756      33        1  0.549964
33     0.159554 -0.150699      34        1  2.551156
34    -0.066742  0.371908      35        1  6.974005
35     0.457336  0.089188      36        1  0.602097
36     0.259359 -0.254986      37        1  4.739025
37     0.361589 -0.312810      38        1  0.833515
38    -0.392113 -0.422705      39        1  0.355672
39     0.061329  0.138901      40        1  2.441299, topic_info=     Category           Freq         Term          Total  loglift  logprob
1007  Default  253639.000000   instructed  253639.000000  30.0000  30.0000
1235  Default  174787.000000     observed  174787.000000  29.0000  29.0000
1641  Default  152161.000000         sink  152161.000000  28.0000  28.0000
1115  Default  186266.000000     maintain  186266.000000  27.0000  27.0000
199   Default  298866.000000          all  298866.000000  26.0000  26.0000
...       ...            ...          ...            ...      ...      ...
1446  Topic40    3311.959545         rear   94397.998070   0.3627  -4.8615
1486  Topic40    3068.473696      replace   76702.644048   0.4939  -4.9378
96    Topic40    2783.327058           32   58634.811771   0.6650  -5.0354
459   Topic40    2719.114722  compartment   61940.644042   0.5868  -5.0587
1279  Topic40    2468.718720          out   24543.390660   1.4159  -5.1553

[2171 rows x 6 columns], token_table=      Topic      Freq    Term
term                         
0        32  0.999757      00
1        12  0.003338     005
1        14  0.000232     005
1        31  0.092215     005
1        32  0.292639     005
...     ...       ...     ...
1995     35  0.257434    yard
1996     11  0.501634  yellow
1996     18  0.079355  yellow
1996     27  0.368432  yellow
1996     40  0.045345  yellow

[6952 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40])

# The ART of Topic Modeling

#### TODO: Now, go back to your favorite LDA model and change the number of topics. 
YOu may want to also run the pyLDAvis to view the results easier
What happens to the topics when you reduce the number of topics  to 5?
What happens when you increase the number of topics to 100? 
Play with the number to get an optimum number of topics. 

In [ ]:
# print a document's text 
corpus[0]
# plot the results from one document 
c_lda_data[0]

# 3.2 Assign a topic to each document

+ for each document, assign the topic (column) with the  highest (max) score from the LDA


In [ ]:
topic_index = np.argmax(lda_data, axis=1)
topic_index.head()

 # 3.3 Assign labels to topics  (More ART)
TODO: replace the maximum topic number with a label chosen by you.  
Try to use the top terms from each topic to give it a label.   
You will need to have the same number of names as you do in your optimal LDA.     
 If you have a LOT of topics, you can just leave them as numbers  

In [ ]:
topic_names = pd.DataFrame(topic_index)

topic_names[topic_names==0] = "0"
topic_names[topic_names==1] = "1"
topic_names[topic_names==2] = "2"
topic_names[topic_names==3] = "3"
topic_names[topic_names==4] = "4"
topic_names[topic_names==5] = "5"
topic_names[topic_names==6] = "6"
topic_names[topic_names==7] = "7"
topic_names[topic_names==8] = "8"

topic_names[topic_names==9] = "9"
topic_names[topic_names==10] = "10"
topic_names[topic_names==11] = "11"

topic_names[topic_names==12] = "12"
topic_names[topic_names==13] = "13"

topic_names[topic_names==14] = "14"
topic_names[topic_names==15] = "15"  
topic_names[topic_names==16] = "16"
topic_names[topic_names==17] = "17"
topic_names[topic_names==18] = "18"
topic_names[topic_names==19] = "19"

In [ ]:
topic_names.head()

# 4. Recommender (optional)
we will use the Ted Talk metadata to add some information to our recommender. 

# import original talks and metadata
merge them on the 'url' column

In [ ]:
ted_trans = pd.read_csv('./data/transcripts.csv', encoding = "UTF-8")  
ted_main = pd.read_csv('./data/ted_main.csv', encoding = "UTF-8")  

In [ ]:
ted_all = pd.merge(ted_trans, right=ted_main, on='url')
ted_all.url = ted_all.url.astype('str',copy=False)

ted_all.head(50)

In [ ]:
def get_recommendations(target_doc, num_of_recs, topics, data, topic_model, vectorizer, topic_model_data):
    
    new_vec = topic_model.transform(
        vectorizer.transform([target_doc]))
    
    nn = NearestNeighbors(n_neighbors=num_of_recs, metric='cosine', algorithm='brute')
    nn.fit(topic_model_data)
    
    results = nn.kneighbors(new_vec)
    
    recommend_list = results[1][0]
    scores = results[0]
                       
    ss = np.array(scores).flat
    for i, resp in enumerate(recommend_list):
        print('\n--- ID ---\n', + resp)
        print('--- distance ---\n', + ss[i])  
        print('--- topic ---')
        print(topics.iloc[resp,0])
        print(data.iloc[resp,1])
        print('--- teds tags ---')
        print(data.iloc[resp,-3])
        
    return recommend_list, ss   

In [ ]:
rec_list, scores = get_recommendations(cleaned_talks[804], num_of_recs=10, topics=topic_names, data=ted_all,
                                       topic_model=lda_model, vectorizer=c_vectorizer, topic_model_data=lda_data)

# 4.1 search and recommend similar documents
We use a great library called fuzzywuzzy to find the titles that is most similar to a search term. then we use this as our target document for the recommendation. 

 #### You will need to install fuzzywuzzy in your conda environment

In [ ]:
from fuzzywuzzy import process
from fuzzywuzzy import fuzz

search_term = "computer science"

titles = ted_all['title']

In [ ]:
tite, score, talk_ind = process.extractOne(search_term, titles, scorer=fuzz.token_set_ratio)

In [ ]:
rec_list, scores = get_recommendations(cleaned_talks[talk_ind], num_of_recs=10, topics=topic_names, data=ted_all,
                                       topic_model=lda_model, vectorizer=c_vectorizer, topic_model_data=lda_data)